In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=10, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Reloading Tuner from trials/hyperband_resnet/tuner0.json

Search: Running Trial #478

Value             |Best Value So Far |Hyperparameter
False             |True              |class_weight
2017_2018_2019    |2017_2018_2019    |training_years
l1l2              |l1l2              |kernel_regularizer
0.5               |0.1               |spatial_dropout
leaky_relu        |relu              |activation
4                 |4                 |pool_size
0.5               |0.5               |dropout
False             |True              |bias_initializer
0.001             |0.0001            |learning_rate
binary_crossent...|binary_crossent...|loss_function
100               |34                |tuner/epochs
34                |12                |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |1                 |tuner/round
0476              |0456              |tuner/trial_id



/home/pj/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 102 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 35/100


I0000 00:00:1724414841.595102    5292 service.cc:145] XLA service 0x7f88c4002920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724414841.595138    5292 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1724414854.898874    5292 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 288s 76ms/step - accuracy: 0.5078 - auc: 0.9151 - loss: 0.3065 - prc: 0.7891 - precision: 0.7580 - recall: 0.6393 - weightedf1score: 0.6610 - weightedf2score: 0.6467 - val_accuracy: 0.5122 - val_auc: 0.9179 - val_loss: 0.3017 - val_prc: 0.8005 - val_precision: 0.7706 - val_recall: 0.6444 - val_weightedf1score: 0.6635 - val_weightedf2score: 0.6501 - learning_rate: 0.0010
Epoch 36/100
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 269s 76ms/step - accuracy: 0.5070 - auc: 0.9142 - loss: 0.3079 - prc: 0.7879 - precision: 0.7589 - recall: 0.6361 - weightedf1score: 0.6581 - weightedf2score: 0.6435 - val_accuracy: 0.5070 - val_auc: 0.9195 - val_loss: 0.2994 - val_prc: 0.8030 - val_precision: 0.7626 - val_recall: 0.6675 - val_weightedf1score: 0.6790 - val_weightedf2score: 0.6714 - learning_rate: 0.0010
Epoch 37/100
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 270s 76ms/step - accuracy: 0.5078 - auc: 0.9144 - loss: 0.3075 - prc: 0.7876 - precision: 0.7577 - recall: 0.6352 - weightedf1score: 0.

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"